In [1]:
from datasets import load_dataset
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from torchvision import transforms
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import roc_curve, auc

c:\Users\maxim\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Importation du dataset**


In [2]:
dataset = load_dataset("Ewakaa/pneumonia_classification_chest_xray")

In [3]:
def display_images(images, labels, title='Images'):
    plt.figure(figsize=(10, 5))
    for i, (image, label) in enumerate(zip(images, labels)):
        plt.subplot(1, len(images), i + 1)
        plt.imshow(image, cmap='gray')
        plt.title(f'Label: {label}')
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

image_label_0 = next(item for item in dataset['train'] if item['label'] == 0)
image_label_1 = next(item for item in dataset['train'] if item['label'] == 1)

display_images([image_label_0['image'], image_label_1['image']], [0, 1], title='Chest X-Ray Images with Labels 0 and 1')

: 

**Distribution des classes**


In [4]:
labels = dataset['train']['label']

label_names = ['Sain' if label == 0 else 'Pneumonie' for label in labels]

label_counts = Counter(label_names)

plt.figure(figsize=(4, 4))
bars = plt.bar(label_counts.keys(), label_counts.values(), color=['blue', 'red'])
plt.title('Distribution des classes')
plt.xlabel('Classe')
plt.ylabel('Nombre d\'échantillons')
plt.xticks(ticks=[0, 1], labels=['Sain', 'Pneumonie'])
plt.show()

print(f'Nombre d\'échantillons de la classe 0 (Sain): {label_counts["Sain"]}')
print(f'Nombre d\'échantillons de la classe 1 (Pneumonie): {label_counts["Pneumonie"]}')

print(f'Nombre total d\'échantillons: {len(labels)}')

: 

**Architecture du modèle d'IA**

In [3]:
class PneumoniaDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]['image']
        label = self.data[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = PneumoniaDataset(dataset['train'], transform=transform)
test_data = PneumoniaDataset(dataset['test'], transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [5]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.fc1 = nn.Linear(128 * 17 * 17, 512)
        self.fc2 = nn.Linear(512, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid() 

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 17 * 17)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = CNNModel()

In [6]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=15):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

In [16]:
train_model(model, train_loader, criterion, optimizer)

torch.save(model.state_dict(), 'model_pneumonie.pth')
print("Model sauvegardé")

Epoch 1/15, Loss: 0.04070255286137581
Epoch 2/15, Loss: 0.02196257068885643
Epoch 3/15, Loss: 0.011530859540485494
Epoch 4/15, Loss: 0.014166022484047858
Epoch 5/15, Loss: 0.012991208080356031
Epoch 6/15, Loss: 0.010213729141923977
Epoch 7/15, Loss: 0.0003929593876708104
Epoch 8/15, Loss: 0.00011691275459835282
Epoch 9/15, Loss: 7.43833714969e-05
Epoch 10/15, Loss: 5.436487531640388e-05
Epoch 11/15, Loss: 4.2421629114774984e-05
Epoch 12/15, Loss: 3.412246515582806e-05
Epoch 13/15, Loss: 2.748569143704027e-05
Epoch 14/15, Loss: 2.3047673988862982e-05
Epoch 15/15, Loss: 2.045126313576784e-05
Model sauvegardé


In [7]:
model.load_state_dict(torch.load('model_pneumonie.pth'))
model.eval()
print('Model chargé avec succès')

Model chargé avec succès


In [8]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            predicted = (outputs.squeeze() > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

evaluate_model(model, test_loader)

Accuracy: 94.15807560137458%


In [9]:
def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0)  
    return image, image_tensor  

In [10]:
def predict_image(model, image_tensor):
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        prediction = (output.item() > 0.5)
    return "Pneumonie" if prediction else "Sain"

In [12]:
image_path = "_Projets/pneumo_IA/dataset/Données/val/PNEUMONIA/person1946_bacteria_4874.jpeg"
image, image_tensor = preprocess_image(image_path, transform)
result = predict_image(model, image_tensor)

#plt.imshow(image)
#plt.axis("off")
#plt.title(f"Résultat : {result}", fontsize=14)
print(f"Résultat : {result}")
#plt.show()

Résultat : Pneumonie


In [22]:
def evaluate_model_roc(model, test_loader):
    model.eval()
    all_labels = []
    all_outputs = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            all_labels.append(labels)
            all_outputs.append(outputs)

    all_labels = torch.cat(all_labels).numpy()
    all_outputs = torch.cat(all_outputs).numpy()

    # Compute ROC curve and ROC area
    fpr, tpr, _ = roc_curve(all_labels, all_outputs)
    roc_auc = auc(fpr, tpr)

    return fpr, tpr, roc_auc

fpr, tpr, roc_auc = evaluate_model_roc(model, test_loader)

In [23]:
import plotly.graph_objs as go
import plotly.express as px

fig = px.line(x=fpr, y=tpr, title='Caractéristiques de fonctionnement du récepteur', labels={'x': 'Taux de faux positifs', 'y': 'Taux de vrai positifs'})

fig.add_shape(type="line", x0=0, y0=0, x1=1, y1=1, line=dict(color="navy", dash="dash"))

fig.update_layout(
    width=600,  
    height=400, 
    xaxis_scaleanchor="y",  
    showlegend=True
)

fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Niveau de chance', line=dict(color='navy', dash='dash')))

fig.show()